# Lab 9: Audio Classification 

## Part 1: Making a speech detector

In this section we will design a simple classifier that will let us know if its input is speech or non-speech. Download the data archive from: [ https://drive.google.com/file/d/1oAnvk-hzzgzZ4di4W0pKw6v3IWLm9u2X/view?usp=sharing ] In this part we will use the dataset in data/SpeechMusic. In it you will find two directories, speech/ and music/ containing data from each class.

Randomly select 50 soundfiles from each directory to use as training data, and use the remaining sounds as testing data. For all of the sounds we will compute a representation that makes the classification easier and we will use a simple Gaussian model to classify them. Do the following:

- Perform an STFT for each sound, take it’s magnitude and raise it to 0.3 to improve contrast
    - We will consider each spctral slice of that to be a data point
- Using the training data of each sound:
    - Calculate the mean column and the diagonal covariance of the columns
    - You will thus get two sets of Gaussian parameters that model each sound class
- For each testing data point:
    - Calculate the likelihood of each column based on the above models
	- To calculate the entire file likelihood add all the frame likelihoods
	- Assign each soundfile to the class that gets the highest likelihood

For extra credit implement the parameter estimation and model likelihood yourself. If you are too lazy for that you can instead use ```sklearn.mixture.GaussianMixture``` to learn a diagonal single-Gaussian model per class.

How do the results look like? If you rerun this with a different training/testing set, is there an appreciable difference? On average over multiple training/testing sets what accuracy do you get?

In [45]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.io.wavfile import read
from scipy import signal
from copy import deepcopy
import math
import random
import einops

def sound( x, rate=8000, label=''):
    from IPython.display import display, Audio, HTML
    if label is '':
        display( Audio( x, rate=rate))
    else:
        display( HTML( 
        '<style> table, th, td {border: 0px; }</style> <table><tr><td>' + label + 
        '</td><td>' + Audio( x, rate=rate)._repr_html_()[3:] + '</td></tr></table>'
        ))
        
def stft(input_sound, dft_size, hop_size, zero_pad, window):
    result = []
    for i in range(0, len(input_sound), hop_size):
        slot = []
        start = i
        end = i + dft_size
        if end >= len(input_sound):
            slot = input_sound[start:]
            zero = np.array([0] * (end - len(input_sound)))
            slot = np.concatenate([slot, zero])
        else:
            slot = input_sound[start:end]
        result.append(np.multiply(np.fft.rfft(np.concatenate([np.multiply(slot, window), np.array(zero_pad)])), (hop_size * 2 / dft_size)))
    return np.array(result)

def show_graph(sub_fig, freq_data, amp_data_len, sample_rate, mode="spectrogram", title=""):
    if mode == "spectrogram":
        transposed_data = np.log(np.absolute(freq_data)).T
        t1 = np.linspace(0, amp_data_len / sample_rate, len(transposed_data[0]))
        t2 = np.linspace(0, sample_rate / 2, len(transposed_data))
        x1, x2 = np.meshgrid(t1, t2)
        sub_fig.pcolormesh(x1, x2, np.array(transposed_data))
        sub_fig.set_title(title)
        
musics = []
speeches = []
data_sr = 22050
print("start")

for i in range(1, 61):
    sr, data = read("data/SpeechMusic/music/%d.wav" % i)
    musics.append(data)
    sr, data = read("data/SpeechMusic/speech/%d.wav" % i)
    speeches.append(data)
    
random.shuffle(musics)
random.shuffle(speeches)

train_musics = musics[:50]
test_musics = musics[50:]
train_speeches = speeches[:50]
test_speeches = speeches[:50]

#plt.plot(train_musics[1])
#sound(data_80, rate=sr_80, label='80s-hi original')

spec_train_musics = []
spec_test_musics = []
spec_train_speeches = []
spec_test_speeches = []

dft_size = 512
hop_size = 512
window = window = np.ones(dft_size)
zero_pad = []

for i in range(50):
    spec = abs(stft(train_musics[i], dft_size, hop_size, zero_pad, window)) ** 3
    #print(spec.shape, np.linalg.norm(spec, axis=1).shape)
    spec /= np.array([np.linalg.norm(spec, axis=1)] * 257).T
    spec_train_musics.append(spec)
    spec = abs(stft(train_speeches[i], dft_size, hop_size, zero_pad, window)) ** 3
    spec /= np.array([np.linalg.norm(spec, axis=1)] * 257).T
    spec_train_speeches.append(spec)
    
for i in range(10):
    spec = abs(stft(test_musics[i], dft_size, hop_size, zero_pad, window)) ** 3
    spec /= np.array([np.linalg.norm(spec, axis=1)] * 257).T
    spec_test_musics.append(spec)
    spec = abs(stft(test_speeches[i], dft_size, hop_size, zero_pad, window)) ** 3
    spec /= np.array([np.linalg.norm(spec, axis=1)] * 257).T
    spec_test_speeches.append(spec)

concat_spec_train_speeches = einops.rearrange(np.array([spec_train_speeches[i] for i in range(50)]), 'h w c -> (h w) c')
concat_spec_train_musics = einops.rearrange(np.array([spec_train_musics[i] for i in range(50)]), 'h w c -> (h w) c')

mean_train_speeches = np.mean(concat_spec_train_speeches, axis=0)
mean_train_musics = np.mean(concat_spec_train_musics, axis=0)
print(concat_spec_train_speeches.shape)
    
cov_train_speeches = (1 / (646 * 50 - 1)) * \
                     np.matmul((concat_spec_train_speeches - np.array([mean_train_speeches] * 32300)).T,
                               (concat_spec_train_speeches - np.array([mean_train_speeches] * 32300)))
cov_train_musics = (1 / (646 * 50 - 1)) * \
                     np.matmul((concat_spec_train_musics - np.array([mean_train_musics] * 32300)).T,
                               (concat_spec_train_musics - np.array([mean_train_musics] * 32300)))

for i in range(10):
    speech_prob = 0
    music_prob = 0
    for j in range(646):
        speech_prob += (-np.log(2 * pow(np.pi, (257/2))) - \
                        0.5 * np.linalg.multi_dot(np.linalg.slogdet(cov_train_speeches)) + \
                        (0.5 * np.linalg.multi_dot([-(spec_test_speeches[i][j] - mean_train_speeches).T, 
                                              np.linalg.inv(cov_train_speeches), 
                                              (spec_test_speeches[i][j] - mean_train_speeches)])))
        music_prob += (-np.log(2 * pow(np.pi, (257/2))) - \
                        0.5 * np.linalg.multi_dot(np.linalg.slogdet(cov_train_musics)) + \
                        (0.5 * np.linalg.multi_dot([-(spec_test_speeches[i][j] - mean_train_musics).T, 
                                              np.linalg.inv(cov_train_musics), 
                                              (spec_test_speeches[i][j] - mean_train_musics)])))
    print("speech %d is classified as" % i, 
          ("speech" if speech_prob >= music_prob else "music"), 
          speech_prob, music_prob)
    speech_prob = 0
    music_prob = 0
    for j in range(646):
        speech_prob += (-np.log(2 * pow(np.pi, (257/2))) - \
                        0.5 * np.linalg.multi_dot(np.linalg.slogdet(cov_train_speeches)) + \
                        (0.5 * np.linalg.multi_dot([-(spec_test_musics[i][j] - mean_train_speeches).T, 
                                              np.linalg.inv(cov_train_speeches), 
                                              (spec_test_musics[i][j] - mean_train_speeches)])))
        music_prob += (-np.log(2 * pow(np.pi, (257/2))) - \
                        0.5 * np.linalg.multi_dot(np.linalg.slogdet(cov_train_musics)) + \
                        (0.5 * np.linalg.multi_dot([-(spec_test_musics[i][j] - mean_train_musics).T, 
                                              np.linalg.inv(cov_train_musics), 
                                              (spec_test_musics[i][j] - mean_train_musics)])))
    print("music %d is classified as" % i, 
          ("speech" if speech_prob >= music_prob else "music"), 
          speech_prob, music_prob)
    
    

start
(32300, 257)
speech 0 is classified as speech 516581.90121303004 -10500553.908094674
music 0 is classified as speech 457829.86439795373 -11181781.787605207
speech 1 is classified as speech 528905.0568253642 517246.35609138367
music 1 is classified as music 528568.587843473 813146.3844146321
speech 2 is classified as speech 512207.48662586545 -6551247.93920757
music 2 is classified as music 555801.437532361 752823.7756091494
speech 3 is classified as speech 565486.5049327619 -688270.9410399741
music 3 is classified as music 555205.1864380435 819550.8868786503
speech 4 is classified as speech 546302.0864633443 -430700.2555742125
music 4 is classified as speech 574136.1223006992 -1359021.2876125686
speech 5 is classified as speech 523356.2101834607 498864.2094676822
music 5 is classified as music 556032.6120386027 642198.0131572182
speech 6 is classified as speech 547023.5458808385 -2882856.6673245244
music 6 is classified as music 566869.0722635087 791489.4341514828
speech 7 is cla

The accuracy is 85%.

## Part 2: Making a music genre classifier

We will repeat the above, but this time we will perform music genre classification. To do so we will use a slightly more elaborate feature representation, and a stronger classification model. If you downloaded the data archive pointed to above, you will find a subset of the CTZAN dataset in the data/genre folder, this is a benchmark data set for music genre classification.

Just as before, you will find a set of directories with examples of each sound class that we want to recognize. For each class, split the soundfiles into a training set (50% of data) and testing set (remaining 50% of data).

For a representation we will use MFCC features. For extra credit, code these yourself otherwise you can use the implementation from the ```librosa``` library. Once all the files are transformed we will have a series of MFCC frames for each recording (as opposed to spectral frames as is in the case of the STFT). We will use these as the data to classify.

For each class learn a Gaussian model (with a diagonal covariance again). This will be the same process as above.
In order to evaluate how good this works we will use the following procedure. For each sound in the training data, get the likelihood of each MFCC frame based on the learned Gaussian models and sum these over the entire file just as we did before. Use the resulting values to get a classification result for each . Report how accurate your results are. Now report the accuracy using your testing data instead.

Now will use a better classifier to hopefully get better accuracy. We will use a Gaussian Mixture Model (```sklearn.mixture.GaussianMixture```). Just as before you should learn one such model for each class using the corresponding training data.

How many Gaussians do you need in your GMM to get the best results? Do the MFCC parameters make a difference? Play around with the numbers to get the best possible results.

In [46]:
#import subprocess
#import sys
#subprocess.check_call([sys.executable, "-m", "pip", "install", "librosa"])

import librosa

classicals = []
discos = []
metals = []
pops = []
reggaes = []

for i in range(100):
    data, sr = librosa.load("data/genres/classical/classical.000%02d.mp3" % i)
    classicals.append(data)
    data, sr = librosa.load("data/genres/disco/disco.000%02d.mp3" % i)
    discos.append(data)
    data, sr = librosa.load("data/genres/metal/metal.000%02d.mp3" % i)
    metals.append(data)
    data, sr = librosa.load("data/genres/pop/pop.000%02d.mp3" % i)
    pops.append(data)
    data, sr = librosa.load("data/genres/reggae/reggae.000%02d.mp3" % i)
    reggaes.append(data)


/home/raymond210129/.local/lib/python3.6/site-packages/librosa/core/audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')
/home/raymond210129/.local/lib/python3.6/site-packages/librosa/core/audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')
/home/raymond210129/.local/lib/python3.6/site-packages/librosa/core/audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')
/home/raymond210129/.local/lib/python3.6/site-packages/librosa/core/audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')
/home/raymond210129/.local/lib/python3.6/site-packages/librosa/core/audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed

/home/raymond210129/.local/lib/python3.6/site-packages/librosa/core/audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')
/home/raymond210129/.local/lib/python3.6/site-packages/librosa/core/audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')
/home/raymond210129/.local/lib/python3.6/site-packages/librosa/core/audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')
/home/raymond210129/.local/lib/python3.6/site-packages/librosa/core/audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')
/home/raymond210129/.local/lib/python3.6/site-packages/librosa/core/audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed

/home/raymond210129/.local/lib/python3.6/site-packages/librosa/core/audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')
/home/raymond210129/.local/lib/python3.6/site-packages/librosa/core/audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')
/home/raymond210129/.local/lib/python3.6/site-packages/librosa/core/audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')
/home/raymond210129/.local/lib/python3.6/site-packages/librosa/core/audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')
/home/raymond210129/.local/lib/python3.6/site-packages/librosa/core/audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed

/home/raymond210129/.local/lib/python3.6/site-packages/librosa/core/audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')
/home/raymond210129/.local/lib/python3.6/site-packages/librosa/core/audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')
/home/raymond210129/.local/lib/python3.6/site-packages/librosa/core/audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')
/home/raymond210129/.local/lib/python3.6/site-packages/librosa/core/audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')
/home/raymond210129/.local/lib/python3.6/site-packages/librosa/core/audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed

/home/raymond210129/.local/lib/python3.6/site-packages/librosa/core/audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')
/home/raymond210129/.local/lib/python3.6/site-packages/librosa/core/audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')
/home/raymond210129/.local/lib/python3.6/site-packages/librosa/core/audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')
/home/raymond210129/.local/lib/python3.6/site-packages/librosa/core/audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')
/home/raymond210129/.local/lib/python3.6/site-packages/librosa/core/audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed

...

/home/raymond210129/.local/lib/python3.6/site-packages/librosa/core/audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')
/home/raymond210129/.local/lib/python3.6/site-packages/librosa/core/audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')
/home/raymond210129/.local/lib/python3.6/site-packages/librosa/core/audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')
/home/raymond210129/.local/lib/python3.6/site-packages/librosa/core/audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')
/home/raymond210129/.local/lib/python3.6/site-packages/librosa/core/audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed

/home/raymond210129/.local/lib/python3.6/site-packages/librosa/core/audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')
/home/raymond210129/.local/lib/python3.6/site-packages/librosa/core/audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')
/home/raymond210129/.local/lib/python3.6/site-packages/librosa/core/audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')
/home/raymond210129/.local/lib/python3.6/site-packages/librosa/core/audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')
/home/raymond210129/.local/lib/python3.6/site-packages/librosa/core/audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed

/home/raymond210129/.local/lib/python3.6/site-packages/librosa/core/audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')
/home/raymond210129/.local/lib/python3.6/site-packages/librosa/core/audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')
/home/raymond210129/.local/lib/python3.6/site-packages/librosa/core/audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')
/home/raymond210129/.local/lib/python3.6/site-packages/librosa/core/audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')
/home/raymond210129/.local/lib/python3.6/site-packages/librosa/core/audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed

/home/raymond210129/.local/lib/python3.6/site-packages/librosa/core/audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')
/home/raymond210129/.local/lib/python3.6/site-packages/librosa/core/audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')
/home/raymond210129/.local/lib/python3.6/site-packages/librosa/core/audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')
/home/raymond210129/.local/lib/python3.6/site-packages/librosa/core/audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')
/home/raymond210129/.local/lib/python3.6/site-packages/librosa/core/audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed

/home/raymond210129/.local/lib/python3.6/site-packages/librosa/core/audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')
/home/raymond210129/.local/lib/python3.6/site-packages/librosa/core/audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')
/home/raymond210129/.local/lib/python3.6/site-packages/librosa/core/audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')
/home/raymond210129/.local/lib/python3.6/site-packages/librosa/core/audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')
/home/raymond210129/.local/lib/python3.6/site-packages/librosa/core/audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed

In [61]:
from sklearn.mixture import GaussianMixture

def sound( x, rate=8000, label=''):
    from IPython.display import display, Audio, HTML
    if label is '':
        display( Audio( x, rate=rate))
    else:
        display( HTML( 
        '<style> table, th, td {border: 0px; }</style> <table><tr><td>' + label + 
        '</td><td>' + Audio( x, rate=rate)._repr_html_()[3:] + '</td></tr></table>'
        ))

train_mfcc_classicals = []
train_mfcc_discos = []
train_mfcc_metals = []
train_mfcc_pops = []
train_mfcc_reggaes = []
test_mfcc_classicals = []
test_mfcc_discos = []
test_mfcc_metals = []
test_mfcc_pops = []
test_mfcc_reggaes = []
hop_length = 512

for i in range(50):
    mfcc = librosa.feature.mfcc(y=classicals[i], sr=sr, hop_length=hop_length, n_mfcc=60)
    train_mfcc_classicals.append(mfcc / np.std(mfcc))# - np.mean(mfcc))
    mfcc = librosa.feature.mfcc(y=discos[i], sr=sr, hop_length=hop_length, n_mfcc=60)
    train_mfcc_discos.append(mfcc / np.std(mfcc))# - np.mean(mfcc))
    mfcc = librosa.feature.mfcc(y=metals[i], sr=sr, hop_length=hop_length, n_mfcc=60)
    train_mfcc_metals.append(mfcc / np.std(mfcc))# - np.mean(mfcc))
    mfcc = librosa.feature.mfcc(y=pops[i], sr=sr, hop_length=hop_length, n_mfcc=60)
    train_mfcc_pops.append(mfcc / np.std(mfcc))# - np.mean(mfcc))
    mfcc = librosa.feature.mfcc(y=reggaes[i], sr=sr, hop_length=hop_length, n_mfcc=60)
    train_mfcc_reggaes.append(mfcc / np.std(mfcc))# - np.mean(mfcc))
for i in range(50, 100):
    mfcc = librosa.feature.mfcc(y=classicals[i], sr=sr, hop_length=hop_length, n_mfcc=60)
    test_mfcc_classicals.append(mfcc / np.std(mfcc))# - np.mean(mfcc))
    mfcc = librosa.feature.mfcc(y=discos[i], sr=sr, hop_length=hop_length, n_mfcc=60)
    test_mfcc_discos.append(mfcc / np.std(mfcc))# - np.mean(mfcc))
    mfcc = librosa.feature.mfcc(y=metals[i], sr=sr, hop_length=hop_length, n_mfcc=60)
    test_mfcc_metals.append(mfcc / np.std(mfcc))# - np.mean(mfcc))
    mfcc = librosa.feature.mfcc(y=pops[i], sr=sr, hop_length=hop_length, n_mfcc=60)
    test_mfcc_pops.append(mfcc / np.std(mfcc))# - np.mean(mfcc))
    mfcc = librosa.feature.mfcc(y=reggaes[i], sr=sr, hop_length=hop_length, n_mfcc=60)
    test_mfcc_reggaes.append(mfcc / np.std(mfcc))# - np.mean(mfcc))

concat_train_classicals = einops.rearrange(np.array([train_mfcc_classicals[i][:,:1290].T for i in range(50)]), 'h w c -> (h w) c')
concat_train_discos = einops.rearrange(np.array([train_mfcc_discos[i][:,:1290].T for i in range(50)]), 'h w c -> (h w) c')
concat_train_metals = einops.rearrange(np.array([train_mfcc_metals[i][:,:1290].T for i in range(50)]), 'h w c -> (h w) c')
concat_train_pops = einops.rearrange(np.array([train_mfcc_pops[i][:,:1290].T for i in range(50)]), 'h w c -> (h w) c')
concat_train_reggaes = einops.rearrange(np.array([train_mfcc_reggaes[i][:,:1290].T for i in range(50)]), 'h w c -> (h w) c')

#print(np.std(concat_train_classicals), np.mean(concat_train_classicals))
#print(np.std(concat_train_discos), np.mean(concat_train_discos))
#print(np.std(concat_train_metals), np.mean(concat_train_metals))
#print(np.std(concat_train_pops), np.mean(concat_train_pops))
#print(np.std(concat_train_reggaes), np.mean(concat_train_reggaes))

print(concat_train_classicals.shape)

n_comp = 2

gmm_classical = GaussianMixture(n_components=n_comp, covariance_type="diag")
gmm_classical.fit(concat_train_classicals)
gmm_disco = GaussianMixture(n_components=n_comp, covariance_type="diag")
gmm_disco.fit(concat_train_discos)
gmm_metal = GaussianMixture(n_components=n_comp, covariance_type="diag")
gmm_metal.fit(concat_train_metals)
gmm_pop = GaussianMixture(n_components=n_comp, covariance_type="diag")
gmm_pop.fit(concat_train_pops)
gmm_reggae = GaussianMixture(n_components=n_comp, covariance_type="diag")
gmm_reggae.fit(concat_train_reggaes)

classical_accuracy = 0
disco_accuracy = 0
metal_accuracy = 0
pop_accuracy = 0
reggae_accuracy = 0

print("test")

for i in range(50):
    
    classical_prob = gmm_classical.score(test_mfcc_classicals[i].T)
    disco_prob = gmm_disco.score(test_mfcc_classicals[i].T)
    metal_prob = gmm_metal.score(test_mfcc_classicals[i].T)
    pop_prob = gmm_pop.score(test_mfcc_classicals[i].T)
    reggae_prob = gmm_reggae.score(test_mfcc_classicals[i].T)
    #print(classical_prob, disco_prob, metal_prob, pop_prob, reggae_prob)
    if max(classical_prob, disco_prob, metal_prob, pop_prob, reggae_prob) == classical_prob:
        classical_accuracy += 0.02
        
    classical_prob = gmm_classical.score(test_mfcc_discos[i].T)
    disco_prob = gmm_disco.score(test_mfcc_discos[i].T)
    metal_prob = gmm_metal.score(test_mfcc_discos[i].T)
    pop_prob = gmm_pop.score(test_mfcc_discos[i].T)
    reggae_prob = gmm_reggae.score(test_mfcc_discos[i].T)
    #print(classical_prob, disco_prob, metal_prob, pop_prob, reggae_prob)
    if max(classical_prob, disco_prob, metal_prob, pop_prob, reggae_prob) == disco_prob:
        disco_accuracy += 0.02
        
    classical_prob = gmm_classical.score(test_mfcc_metals[i].T)
    disco_prob = gmm_disco.score(test_mfcc_metals[i].T)
    metal_prob = gmm_metal.score(test_mfcc_metals[i].T)
    pop_prob = gmm_pop.score(test_mfcc_metals[i].T)
    reggae_prob = gmm_reggae.score(test_mfcc_metals[i].T)
    #print(classical_prob, disco_prob, metal_prob, pop_prob, reggae_prob)
    if max(classical_prob, disco_prob, metal_prob, pop_prob, reggae_prob) == metal_prob:
        metal_accuracy += 0.02
        
    classical_prob = gmm_classical.score(test_mfcc_pops[i].T)
    disco_prob = gmm_disco.score(test_mfcc_pops[i].T)
    metal_prob = gmm_metal.score(test_mfcc_pops[i].T)
    pop_prob = gmm_pop.score(test_mfcc_pops[i].T)
    reggae_prob = gmm_reggae.score(test_mfcc_pops[i].T)
    #print(classical_prob, disco_prob, metal_prob, pop_prob, reggae_prob)
    if max(classical_prob, disco_prob, metal_prob, pop_prob, reggae_prob) == pop_prob:
        pop_accuracy += 0.02
        
    classical_prob = gmm_classical.score(test_mfcc_reggaes[i].T)
    disco_prob = gmm_disco.score(test_mfcc_reggaes[i].T)
    metal_prob = gmm_metal.score(test_mfcc_reggaes[i].T)
    pop_prob = gmm_pop.score(test_mfcc_reggaes[i].T)
    reggae_prob = gmm_reggae.score(test_mfcc_reggaes[i].T)
    #print(classical_prob, disco_prob, metal_prob, pop_prob, reggae_prob)
    if max(classical_prob, disco_prob, metal_prob, pop_prob, reggae_prob) == reggae_prob:
        reggae_accuracy += 0.02
        
print("classical accuracy:", classical_accuracy)
print("disco accuracy:", disco_accuracy)
print("metal accuracy:", metal_accuracy)
print("pop accuracy:", pop_accuracy)
print("reggae accuracy:", reggae_accuracy)

    

(64500, 60)
test
classical accuracy: 0.8600000000000004
disco accuracy: 0.04
metal accuracy: 0.9000000000000005
pop accuracy: 0.8800000000000004
reggae accuracy: 0.4200000000000001


1. The accuracy is especially low on predicting reggae musics. 
2. There isn't significant difference among different n_components:
    - n_comp=2:
        - classical accuracy: 0.8600000000000004
        - disco accuracy: 0.04
        - metal accuracy: 0.9000000000000005
        - pop accuracy: 0.8800000000000004
        - reggae accuracy: 0.4200000000000001
    - n_comp=3
        - classical accuracy: 0.8600000000000004
        - disco accuracy: 0.02
        - metal accuracy: 0.9200000000000005
        - pop accuracy: 0.8400000000000004
        - reggae accuracy: 0.4000000000000001
    - n_comp=4
        - 0.8600000000000004
        - disco accuracy: 0
        - metal accuracy: 0.9200000000000005
        - pop accuracy: 0.8400000000000004
        - reggae accuracy: 0.4200000000000001


## Part 3: Make it better (extra credit, required for 4-hour registrants)

There is no shortage of techniques (and free code) to use for classification. Revisit the two problems above and use any other type of classifier you want (Neural Nets, Boosting, Decision Trees, whatever). Also feel free to use any feature you want. Can you improve on the results you got before? How much higher can you get your accuracy for either case?